In [ ]:
# Change working directory to project root.
import os
if os.getcwd().endswith('training'):
    print(os.getcwd())
    os.chdir('../..')
print(os.getcwd())

In [ ]:
# Test whether progress bars work.
from tqdm.auto import tqdm
_ = list(tqdm(range(1)))

In [ ]:
import awe.model.eval
import awe.training.trainer
import awe.utils
awe.utils.reload('awe.model', 'awe.training')

In [ ]:
params = awe.training.params.Params.load_user(normalize=True)
params

In [ ]:
# Preserve previously loaded pretrained weights for faster iteration.
pretrained = trainer.pretrained if 'trainer' in globals() else None

In [ ]:
trainer = awe.training.trainer.Trainer(params)

In [ ]:
# Reload evaluator.
trainer.evaluator = awe.model.eval.Evaluator(trainer.label_map)

In [ ]:
trainer.create_version()

In [ ]:
if pretrained is not None:
    trainer.pretrained = pretrained
trainer.load_pretrained()

In [ ]:
trainer.load_data()

In [ ]:
trainer.create_model()

In [ ]:
trainer.create_writer()

In [ ]:
trainer.train()

In [ ]:
trainer.validate(awe.training.trainer.RunInput(trainer.create_dataloader(trainer.val_pages)))

In [ ]:
[p.file_path for p in trainer.val_pages]

In [ ]:
preds = trainer.predict(awe.training.trainer.RunInput(trainer.create_dataloader(trainer.val_pages)))
trainer.decode(preds)